In [1]:
# import dependencies
import requests
import pandas as pd
import time
from pprint import pprint


In [2]:
# import csv
housing_data_df = pd.read_csv("Resources/housing.csv")
housing_data_df.head()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [6]:
# grab lat,lon coordinates from housing_data_df
coordinates = housing_data_df[["latitude", "longitude"]]

round_coordinates = coordinates.round(1)
unique_coordinates = round_coordinates.groupby(["latitude", "longitude"]).size().reset_index()

lat = unique_coordinates["latitude"]
lon = unique_coordinates["longitude"]
lat_lon = list(zip(lat, lon))

len(lat_lon)


1563

In [7]:
# convert lat,lon coordinate to county

county_data = []

line = "-" *30

print(line)
print("Begin Retrieval")
print(line)

throttle = 25

# get county information for each coordinate
for i in range(len(lat_lon)):
    url = f"https://geo.fcc.gov/api/census/area?lat={lat_lon[i][0]}&lon={lat_lon[i][1]}&format=json"
    response = requests.get(url).json()

    coord = {
        "latitude" : lat_lon[i][0],
        "longitude" : lat_lon[i][1]
    }
    
    try:
        print(f"Request {i+1}/{len(lat_lon)} Retrieving data for {lat_lon[i]}")
        
        # remove bbox info from response
        data = response["results"][0]
        data.pop("bbox", None)

        # append lat,lon dictionary with coordinate data
        coord.update(data)

    except IndexError:
        print("Data not found...")

    # append info to county list
    county_data.append(coord)
    
    # stop calls for 5 sec after throttle_count
    if i % throttle == throttle-1:
        time.sleep(10)

print(line)
print("Finished Retrieval")
print(line)


------------------------------
Begin Retrieval
------------------------------
Request 1/1563 Retrieving data for (32.5, -117.0)
Data not found...
Request 2/1563 Retrieving data for (32.6, -117.2)
Request 3/1563 Retrieving data for (32.6, -117.1)
Request 4/1563 Retrieving data for (32.6, -117.0)


KeyboardInterrupt: 

In [5]:
len(county_data)

714

In [ ]:
# store data in dataframe
county_data_df = pd.DataFrame(county_data)
county_data_df.head(20)


In [ ]:
# combine data with corresponding coordinates
# coord_county_df = pd.concat([coordinates, county_data_df], axis=1)

# save to csv
coord_county_df = county_data_df.to_csv("Resources/county_data.csv")

# preview dataframe
coord_county_df
